# Load libraries and set the coordinate system

In [1]:
%pylab inline

import numpy as np
import cvxpy
from cvxpy import Variable, Minimize, sum_squares, norm, Problem, Parameter, mul_elemwise, sum_entries, Constant
from scipy import sparse
import sys, argparse
from scipy import ndimage
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys

sys.path.append('../../')
from tractionforce.elasticity import *
import gc
from tractionforce.norms import *
from joblib import Parallel, delayed
import multiprocessing

"""
Set up the domain and the grid
"""

x_obs_positions = np.arange(100)
y_obs_positions = np.arange(100)

dx = abs(x_obs_positions[1]-x_obs_positions[0])
dy = abs(y_obs_positions[1]-y_obs_positions[0])

N = len(x_obs_positions)
M = len(y_obs_positions)

coords = [[ [x,y] for y in y_obs_positions] for x in x_obs_positions]
coords = np.array([item for sublist in coords for item in sublist])
distances = ((coords[:,0]-50)**2 + (coords[:,1]-50)**2)**0.5
angles = np.arctan2(coords[:,1],coords[:,0])

a = 30
b = 35 # stress support
c = 40 # cell boundary

# build the stress field




Populating the interactive namespace from numpy and matplotlib


# Generate the stress field

In [ ]:
CUTOFF  = 16

fr = 1e-4
ftheta = 1e-4

m = 2
n = 4

condition_outside = (distances > c) * (distances < c + CUTOFF)
condition_inside = distances < c

stress_r = 0 + (distances>=a) * (distances<=35) * fr * np.sin(m*angles)
stress_theta = 0 + (distances>=a) * (distances<=35) * ftheta * np.cos(n*angles)

stress_x = stress_r*cos(angles) + stress_theta*sin(angles)
stress_y = stress_r*sin(angles) + stress_theta*cos(angles)

s_x = stress_x[condition_inside]
s_y = stress_y[condition_inside]

# rotate to x,y 

# Generate linear system matrices

In [ ]:

x_out = np.array(coords[condition_outside,0]/dx,dtype=int)
y_out = np.array(coords[condition_outside,1]/dy,dtype=int)

x_in = np.array(coords[condition_inside,0]/dx,dtype=int)
y_in = np.array(coords[condition_inside,1]/dy,dtype=int)

x_center = np.mean(x_in)
y_center = np.mean(y_in)


n_in = len(x_in)
n_out = len(x_out)

print("Size of the problem is " + str( n_in + n_out))

deltax_in_in = x_in[...,np.newaxis] - x_in[np.newaxis, ...]  # should be x-x'
deltax_out_in = x_out[...,np.newaxis] - x_in[np.newaxis, ...]  # should be x-x'
deltay_in_in = y_in[...,np.newaxis] - y_in[np.newaxis, ...]  # y - y'
deltay_out_in = y_out[...,np.newaxis] - y_in[np.newaxis,...] # y - y'

l2_in_plus_in_plus = (np.array([deltax_in_in*dx - dx/2.0, deltay_in_in*dy - dy/2.0])**2).sum(axis=0)**0.5
l2_in_plus_in_minus = (np.array([deltax_in_in*dx - dx/2.0, deltay_in_in*dy + dy/2.0])**2).sum(axis=0)**0.5
l2_in_minus_in_plus = (np.array([deltax_in_in*dx + dx/2.0, deltay_in_in*dy - dy/2.0])**2).sum(axis=0)**0.5
l2_in_minus_in_minus = (np.array([deltax_in_in*dx + dx/2.0, deltay_in_in*dy+ dy/2.0]) ** 2).sum(axis=0) ** 0.5

l2_out_plus_in_plus = (np.array([deltax_out_in*dx - dx/2.0, deltay_out_in*dy - dy/2.0])**2).sum(axis=0)**0.5
l2_out_plus_in_minus = (np.array([deltax_out_in*dx - dx/2.0, deltay_out_in*dy + dy/2.0])**2).sum(axis=0)**0.5
l2_out_minus_in_plus = (np.array([deltax_out_in*dx + dx/2.0, deltay_out_in*dy - dy/2.0])**2).sum(axis=0)**0.5
l2_out_minus_in_minus = (np.array([deltax_out_in*dx + dx/2.0, deltay_out_in*dy + dy/2.0]) ** 2).sum(axis=0) ** 0.5

x_adjacency = sparse.csr_matrix((deltax_in_in == -1)*(deltay_in_in == 0)*-1 + (deltax_in_in == 1)*(deltay_in_in == 0)*1)
y_adjacency = sparse.csr_matrix((deltay_in_in == -1)*(deltax_in_in == 0)*-1 + (deltay_in_in == 1)*(deltax_in_in == 0)*1)

A_in_in_x = fxx(deltax_in_in*dx-dx/2. , deltay_in_in*dy-dy/2.0 , l2_in_plus_in_plus) - \
          fxx(deltax_in_in*dx-dx/2. , deltay_in_in*dy+dy/2.0, l2_in_plus_in_minus) -\
          fxx(deltax_in_in*dx+dx/2. , deltay_in_in*dy-dy/2.0, l2_in_minus_in_plus) + \
          fxx(deltax_in_in*dx+dx/2. , deltay_in_in*dy+dy/2.0, l2_in_minus_in_minus)

A_out_in_x = fxx(deltax_out_in*dx-dx/2. , deltay_out_in*dy-dy/2.0 , l2_out_plus_in_plus) - \
          fxx(deltax_out_in*dx-dx/2. , deltay_out_in*dy+dy/2.0, l2_out_plus_in_minus) -\
          fxx(deltax_out_in*dx+dx/2. , deltay_out_in*dy-dy/2.0, l2_out_minus_in_plus) + \
          fxx(deltax_out_in*dx+dx/2. , deltay_out_in*dy+dy/2.0, l2_out_minus_in_minus)

D_in_in_x = fxy(deltax_in_in*dx-dx/2. , deltay_in_in*dy-dy/2.0 , l2_in_plus_in_plus) - \
          fxy(deltax_in_in*dx-dx/2. , deltay_in_in*dy+dy/2.0, l2_in_plus_in_minus) - \
          fxy(deltax_in_in*dx+dx/2. , deltay_in_in*dy-dy/2.0, l2_in_minus_in_plus) + \
          fxy(deltax_in_in*dx+dx/2. , deltay_in_in*dy+dy/2.0, l2_in_minus_in_minus)

D_out_in_x = fxy(deltax_out_in*dx-dx/2. , deltay_out_in*dy-dy/2.0 , l2_out_plus_in_plus) - \
           fxy(deltax_out_in*dx-dx/2. , deltay_out_in*dy+dy/2.0, l2_out_plus_in_minus) - \
           fxy(deltax_out_in*dx+dx/2. , deltay_out_in*dy-dy/2.0, l2_out_minus_in_plus) + \
           fxy(deltax_out_in*dx+dx/2. , deltay_out_in*dy+dy/2.0, l2_out_minus_in_minus)

B_in_in_x = x_in[..., np.newaxis]*A_in_in_x - fxxx(deltax_in_in-dx/2. , deltay_in_in-dy/2.0 , l2_in_plus_in_plus) + \
              fxxx(deltax_in_in-dx/2. , deltay_in_in+dy/2.0, l2_in_plus_in_minus) +\
              fxxx(deltax_in_in+dx/2. , deltay_in_in-dy/2.0, l2_in_minus_in_plus) - \
              fxxx(deltax_in_in+dx/2. , deltay_in_in+dy/2.0, l2_in_minus_in_minus)

B_out_in_x = x_out[..., np.newaxis]*A_out_in_x - fxxx(deltax_out_in-dx/2. , deltay_out_in-dy/2.0 , l2_out_plus_in_plus) + \
          fxxx(deltax_out_in-dx/2. , deltay_out_in+dy/2.0, l2_out_plus_in_minus) +\
          fxxx(deltax_out_in+dx/2. , deltay_out_in-dy/2.0, l2_out_minus_in_plus) - \
          fxxx(deltax_out_in+dx/2. , deltay_out_in+dy/2.0, l2_out_minus_in_minus)

C_in_in_x = y_in[..., np.newaxis]*A_in_in_x - fxxy(deltax_in_in-dx/2. , deltay_in_in-dy/2.0 , l2_in_plus_in_plus) + \
          fxxy(deltax_in_in-dx/2. , deltay_in_in+dy/2.0, l2_in_plus_in_minus) + \
          fxxy(deltax_in_in+dx/2. , deltay_in_in-dy/2.0, l2_in_minus_in_plus) - \
          fxxy(deltax_in_in+dx/2. , deltay_in_in+dy/2.0, l2_in_minus_in_minus)

C_out_in_x = y_out[..., np.newaxis]*A_out_in_x - fxxy(deltax_out_in-dx/2. , deltay_out_in-dy/2.0 , l2_out_plus_in_plus) + \
           fxxy(deltax_out_in-dx/2. , deltay_out_in+dy/2.0, l2_out_plus_in_minus) + \
           fxxy(deltax_out_in+dx/2. , deltay_out_in-dy/2.0, l2_out_minus_in_plus) - \
           fxxy(deltax_out_in+dx/2. , deltay_out_in+dy/2.0, l2_out_minus_in_minus)

E_in_in_x = x_in[..., np.newaxis] * D_in_in_x - fxyx(deltax_in_in - dx / 2., deltay_in_in - dy / 2.0,
                                                 l2_in_plus_in_plus) + \
          fxyx(deltax_in_in - dx / 2., deltay_in_in + dy / 2.0, l2_in_plus_in_minus) + \
          fxyx(deltax_in_in + dx / 2., deltay_in_in - dy / 2.0, l2_in_minus_in_plus) - \
          fxyx(deltax_in_in + dx / 2., deltay_in_in + dy / 2.0, l2_in_minus_in_minus)

E_out_in_x = x_out[..., np.newaxis] * D_out_in_x - fxyx(deltax_out_in - dx / 2., deltay_out_in - dy / 2.0,
                                                   l2_out_plus_in_plus) + \
           fxyx(deltax_out_in - dx / 2., deltay_out_in + dy / 2.0, l2_out_plus_in_minus) + \
           fxyx(deltax_out_in + dx / 2., deltay_out_in - dy / 2.0, l2_out_minus_in_plus) - \
           fxyx(deltax_out_in + dx / 2., deltay_out_in + dy / 2.0, l2_out_minus_in_minus)

F_in_in_x = y_in[..., np.newaxis]*D_in_in_x - fxyx(deltax_in_in-dx/2. , deltay_in_in-dy/2.0 , l2_in_plus_in_plus) + \
          fxyx(deltax_in_in-dx/2. , deltay_in_in+dy/2.0, l2_in_plus_in_minus) + \
          fxyx(deltax_in_in+dx/2. , deltay_in_in-dy/2.0, l2_in_minus_in_plus) - \
          fxyx(deltax_in_in+dx/2. , deltay_in_in+dy/2.0, l2_in_minus_in_minus)

F_out_in_x = y_out[..., np.newaxis]* D_out_in_x - fxyx(deltax_out_in-dx/2. , deltay_out_in-dy/2.0 , l2_out_plus_in_plus) + \
           fxyx(deltax_out_in-dx/2. , deltay_out_in+dy/2.0, l2_out_plus_in_minus) + \
           fxyx(deltax_out_in+dx/2. , deltay_out_in-dy/2.0, l2_out_minus_in_plus) - \
           fxyx(deltax_out_in+dx/2. , deltay_out_in+dy/2.0, l2_out_minus_in_minus)
            
# u_y measurements      
            
A_in_in_y = fxx( deltay_in_in*dy-dy/2.0 , deltax_in_in*dx-dx/2. , l2_in_plus_in_plus) - \
          fxx(deltay_in_in*dy+dy/2.0, deltax_in_in*dx-dx/2. , l2_in_plus_in_minus) -\
          fxx(deltay_in_in*dy-dy/2.0, deltax_in_in*dx+dx/2. , l2_in_minus_in_plus) + \
          fxx(deltay_in_in*dy+dy/2.0, deltax_in_in*dx+dx/2. , l2_in_minus_in_minus)

A_out_in_y = fxx(deltay_out_in*dy-dy/2.0 , deltax_out_in*dx-dx/2. , l2_out_plus_in_plus) - \
          fxx(deltay_out_in*dy+dy/2.0, deltax_out_in*dx-dx/2. , l2_out_plus_in_minus) -\
          fxx(deltay_out_in*dy-dy/2.0, deltax_out_in*dx+dx/2. , l2_out_minus_in_plus) + \
          fxx(deltay_out_in*dy+dy/2.0, deltax_out_in*dx+dx/2. , l2_out_minus_in_minus)

D_in_in_y = fxy(deltay_in_in*dy-dy/2.0 , deltax_in_in*dx-dx/2. , l2_in_plus_in_plus) - \
          fxy(deltay_in_in*dy+dy/2.0, deltax_in_in*dx-dx/2. , l2_in_plus_in_minus) - \
          fxy(deltay_in_in*dy-dy/2.0, deltax_in_in*dx+dx/2. , l2_in_minus_in_plus) + \
          fxy(deltay_in_in*dy+dy/2.0, deltax_in_in*dx+dx/2. , l2_in_minus_in_minus)

D_out_in_y = fxy(deltay_out_in*dy-dy/2.0 , deltax_out_in*dx-dx/2. , l2_out_plus_in_plus) - \
           fxy(deltay_out_in*dy+dy/2.0, deltax_out_in*dx-dx/2. , l2_out_plus_in_minus) - \
           fxy(deltay_out_in*dy-dy/2.0, deltax_out_in*dx+dx/2. , l2_out_minus_in_plus) + \
           fxy(deltay_out_in*dy+dy/2.0, deltax_out_in*dx+dx/2. , l2_out_minus_in_minus)

B_in_in_y = y_in[..., np.newaxis]*A_in_in_y - fxxx(deltay_in_in-dy/2.0 , deltax_in_in-dx/2. , l2_in_plus_in_plus) + \
              fxxx(deltay_in_in+dy/2.0, deltax_in_in-dx/2. , l2_in_plus_in_minus) +\
              fxxx(deltay_in_in-dy/2.0, deltax_in_in+dx/2. , l2_in_minus_in_plus) - \
              fxxx(deltay_in_in+dy/2.0, deltax_in_in+dx/2. , l2_in_minus_in_minus)

B_out_in_y = y_out[..., np.newaxis]*A_out_in_y - fxxx(deltay_out_in-dy/2.0 , deltax_out_in-dx/2. , l2_out_plus_in_plus) + \
          fxxx(deltay_out_in+dy/2.0, deltax_out_in-dx/2. , l2_out_plus_in_minus) +\
          fxxx(deltay_out_in-dy/2.0, deltax_out_in+dx/2. , l2_out_minus_in_plus) - \
          fxxx(deltay_out_in+dy/2.0, deltax_out_in+dx/2. , l2_out_minus_in_minus)

C_in_in_y = x_in[..., np.newaxis]*A_in_in_y - fxxy(deltay_in_in-dy/2.0 , deltax_in_in-dx/2. , l2_in_plus_in_plus) + \
          fxxy(deltay_in_in+dy/2.0, deltax_in_in-dx/2. , l2_in_plus_in_minus) + \
          fxxy(deltay_in_in-dy/2.0, deltax_in_in+dx/2. , l2_in_minus_in_plus) - \
          fxxy(deltay_in_in+dy/2.0, deltax_in_in+dx/2. , l2_in_minus_in_minus)

C_out_in_y = x_out[..., np.newaxis]*A_out_in_y - fxxy(deltay_out_in-dy/2.0, deltax_out_in-dx/2., l2_out_plus_in_plus) + \
           fxxy(deltay_out_in+dy/2.0, deltax_out_in-dx/2. , l2_out_plus_in_minus) + \
           fxxy(deltay_out_in-dy/2.0, deltax_out_in+dx/2. , l2_out_minus_in_plus) - \
           fxxy(deltay_out_in+dy/2.0, deltax_out_in+dx/2. , l2_out_minus_in_minus)

E_in_in_y = y_in[..., np.newaxis] * D_in_in_y - fxyx(deltay_in_in - dy / 2.0, deltax_in_in - dx / 2., 
                                                 l2_in_plus_in_plus) + \
          fxyx(deltay_in_in + dy / 2.0, deltax_in_in - dx / 2., l2_in_plus_in_minus) + \
          fxyx(deltay_in_in - dy / 2.0, deltax_in_in + dx / 2., l2_in_minus_in_plus) - \
          fxyx(deltay_in_in + dy / 2.0, deltax_in_in + dx / 2., l2_in_minus_in_minus)

E_out_in_y = y_out[..., np.newaxis] * D_out_in_y - fxyx(deltay_out_in - dy / 2.0, deltax_out_in - dx / 2., 
                                                   l2_out_plus_in_plus) + \
           fxyx(deltay_out_in + dy / 2.0, deltax_out_in - dx / 2., l2_out_plus_in_minus) + \
           fxyx(deltay_out_in - dy / 2.0, deltax_out_in + dx / 2., l2_out_minus_in_plus) - \
           fxyx(deltay_out_in + dy / 2.0, deltax_out_in + dx / 2., l2_out_minus_in_minus)

F_in_in_y = x_in[..., np.newaxis]*D_in_in_y - fxyx(deltay_in_in-dy/2.0 , deltax_in_in-dx/2. , l2_in_plus_in_plus) + \
          fxyx(deltay_in_in+dy/2.0, deltax_in_in-dx/2. , l2_in_plus_in_minus) + \
          fxyx(deltay_in_in-dy/2.0, deltax_in_in+dx/2. , l2_in_minus_in_plus) - \
          fxyx(deltay_in_in+dy/2.0, deltax_in_in+dx/2. , l2_in_minus_in_minus)

F_out_in_y = x_out[..., np.newaxis]* D_out_in_y - fxyx(deltay_out_in-dy/2.0 , deltax_out_in-dx/2. , l2_out_plus_in_plus) + \
           fxyx(deltay_out_in+dy/2.0, deltax_out_in-dx/2. , l2_out_plus_in_minus) + \
           fxyx(deltay_out_in-dy/2.0, deltax_out_in+dx/2. , l2_out_minus_in_plus) - \
           fxyx(deltay_out_in+dy/2.0, deltax_out_in+dx/2. , l2_out_minus_in_minus)


G_in_in_xx = A_in_in_x + B_in_in_x + C_in_in_x
G_in_in_xy = D_in_in_x + E_in_in_x + F_in_in_x
G_out_in_xx = (A_out_in_x + B_out_in_x + C_out_in_x)
G_out_in_xy = (D_out_in_x + E_out_in_x + F_out_in_x)

G_in_in_yy = A_in_in_y + B_in_in_y + C_in_in_y
G_in_in_yx = D_in_in_y + E_in_in_y + F_in_in_y
G_out_in_yy = (A_out_in_y + B_out_in_y + C_out_in_y)
G_out_in_yx = (D_out_in_y + E_out_in_y + F_out_in_y)

del A_in_in_x, B_in_in_x, C_in_in_x, D_in_in_x, E_in_in_x, F_in_in_x
del A_out_in_x, B_out_in_x, C_out_in_x, D_out_in_x, E_out_in_x, F_out_in_x
del A_in_in_y, B_in_in_y, C_in_in_y, D_in_in_y, E_in_in_y, F_in_in_y
del A_out_in_y, B_out_in_y, C_out_in_y, D_out_in_y, E_out_in_y, F_out_in_y

gc.collect()

Size of the problem is 9027


In [ ]:
# Generate the measured dispalcements

u_in_x = G_in_in_xx.dot(s_x) + G_in_in_xy.dot(s_y)
u_out_x = G_out_in_xx.dot(s_x) + G_out_in_xy.dot(s_y)

u_in_y = G_in_in_yx.dot(s_x) + G_in_in_yy.dot(s_y)
u_out_y = G_out_in_yx.dot(s_x)  + G_out_in_yy.dot(s_y)


In [ ]:
# make derivative matrices Lx Ly

Dx = sparse.csr_matrix((deltax_in_in == 0)*(deltay_in_in == 0)*-1 + (deltax_in_in == 1)*(deltay_in_in == 0)*1)
rowsums = np.squeeze(np.asarray((Dx.sum(axis=1) != 0)))
Dx[rowsums,:] = 0
Dx.eliminate_zeros()
Dx = Constant(Dx)

Dy = sparse.csr_matrix(
    (deltay_in_in == 0) * (deltax_in_in == 0) * -1 + (deltay_in_in == 1) * (deltax_in_in == 0) * 1)
rowsums = np.squeeze(np.asarray((Dy.sum(axis=1) != 0)))
Dy[rowsums,:] = 0
Dy.eliminate_zeros()
Dy = Constant(Dy)

del deltax_in_in, deltay_in_in, deltax_out_in, deltay_out_in
del l2_in_plus_in_plus, l2_in_plus_in_minus, l2_in_minus_in_plus, l2_in_minus_in_minus
del l2_out_plus_in_plus, l2_out_plus_in_minus, l2_out_minus_in_plus, l2_out_minus_in_minus
gc.collect()



In [ ]:
N_SOLUTIONS = 24
gamma_vals = np.logspace(-5, 0, N_SOLUTIONS)


# Coords are 1:100 in each direction
# annulus centered
# inner radius 20
# outer radius 30
# boundary radius 35

gamma = Parameter(sign="positive",value=1)

sigma_xz = Variable(n_in)
sigma_yz = Variable(n_in)
predicted_in_x = G_in_in_xx * sigma_xz + G_in_in_xy * sigma_yz
predicted_out_x = G_out_in_xx * sigma_xz + G_out_in_xy * sigma_yz
predicted_in_y = G_in_in_yx * sigma_xz + G_in_in_yy * sigma_yz
predicted_out_y = G_out_in_yx * sigma_xz + G_out_in_yy * sigma_yz


error = sum_squares(u_x_in - predicted_in_x) + sum_squares(u_x_out - predicted_out_x) + \
    sum_squares(u_y_in - predicted_in_y) + sum_squares(u_y_out - predicted_out_y)

forceconstraints = [sum_entries(sigma_xz)==0, sum_entries(sigma_yz)==0] # add torque-free constraint here
net_torque = sum_entries(mul_elemwise(x_in-x_center,sigma_yz) - mul_elemwise(y_in-y_center,sigma_xz))

torqueconstraints = [net_torque == 0]

constraints = forceconstraints + torqueconstraints

In [ ]:
regularity_penalty = norm(sigma_xz+sigma_yz,p=1)
objective = Minimize(error + gamma*regularity_penalty)
prob = Problem(objective, constraints)

sq_penalty = []
l1_penalty = []
sigma_xz_values = []
sigma_yz_values = []

u_x_values = []
u_y_values = []

In [ ]:
num_cores = multiprocessing.cpu_count()

def invert_one(val):
    gamma.value = val
    try:
        prob.solve(verbose= False, max_iters = 50,
                   warm_start=True, solver = cvxpy.ECOS,
                   feastol = 1e-6, reltol = 1e-5,
                   abstol = 1e-6)

    except cvxpy.SolverError:
        return None

    sq_penalty.append(error.value)
    l1_penalty.append(regularity_penalty.value)
    sigma_xz_values.append(sigma_xz.value)
    sigma_yz_values.append(sigma_yz.value)

    force = np.zeros_like(coords)
    force[condition_inside,0] = sigma_xz.value.reshape((n_in,))
    force[condition_inside,1] = sigma_yz.value.reshape((n_in,))

    u_x = np.zeros(coords.shape[0])
    u_y = np.zeros(coords.shape[0])
    u_x[condition_inside] = predicted_in_x.value
    u_x[condition_outside] = predicted_out_x.value
    u_y[condition_inside] = predicted_in_y.value
    u_y[condition_outside] = predicted_out_y.value
    #u_x_values.append(u_x)

    return force, u_x, u_y, error.value, regularity_penalty.value, val


results = Parallel(n_jobs=num_cores)(delayed(invert_one)(val) for val in gamma_vals)
